In [1]:
import os

import numpy as np
import torch

from envs import make_vec_envs_ViZDoom
from model import Policy

from time import sleep

In [2]:
result_dir = "../vizdoom_record/"
MODEL_SAVE_PATH = os.path.join(result_dir, "model.save")

In [3]:
seed = 99
recurrent_policy = False
cuda = False
device = torch.device("cuda:0" if cuda else "cpu")

In [4]:
envs = make_vec_envs_ViZDoom(seed, 1, device, render=True)

/Users/harry/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/Users/harry/venv/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [5]:
actor_critic = Policy(envs.observation_space.shape, envs.action_space,
    base_kwargs={'recurrent': recurrent_policy})
actor_critic.to(device)
actor_critic.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location='cpu'))

In [6]:
masks = torch.zeros(1, 1)
obs = envs.reset()

while True:
    with torch.no_grad():
        value, action, _, recurrent_hidden_states = actor_critic.act(
            obs, None, masks)
    
    obs, reward, done, _ = envs.step(action)

    masks.fill_(0.0 if done else 1.0)
    
    sleep(1.0/120)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/harry/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-235cac1535d1>", line 9, in <module>
    obs, reward, done, _ = envs.step(action)
  File "/Users/harry/F18/Multimodal/vizdoom_agent/openai.py", line 79, in step
    return self.step_wait()
  File "/Users/harry/F18/Multimodal/vizdoom_agent/envs.py", line 167, in step_wait
    obs, reward, done, info = self.venv.step_wait()
  File "/Users/harry/F18/Multimodal/vizdoom_agent/openai.py", line 226, in step_wait
    results = [remote.recv() for remote in self.remotes]
  File "/Users/harry/F18/Multimodal/vizdoom_agent/openai.py", line 226, in <listcomp>
    results = [remote.recv() for remote in self.remotes]
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/Librar

KeyboardInterrupt: 